Let's walk through some of this uarray code to see if it makes sense

In [1]:
import uarray

In order to get a literal array in, we first just have it as a scalar with the value being a tuple of (shape, data), where data is the array as nested tuples.

In [2]:
a = uarray.Array((2, 2), (1, 2, 3, 4))
print(repr(a))
print(a)

Array('[1 2 3 4; ρ=(2, 2)]')
[1 2 3 4; ρ=(2, 2)]


Now let's see if we can get the shape of that :

In [3]:
s = uarray.Shape(a)
print(s)
print(uarray.replace(s))

ρ([1 2 3 4; ρ=(2, 2)])
<2 2>


This is a bit more of an involved replacement. Let's see how it works by iterating over the `max_count` of replacements:

Now let's turn this back into an array

In [4]:
a2 = uarray.AsArray(s)
a2

AsArray(Shape(Array('[1 2 3 4; ρ=(2, 2)]')))

In [5]:
for i in range(12):
    print(uarray.replace(a2, i))

AsArray(ρ([1 2 3 4; ρ=(2, 2)]))
AsArrayWithShape(ρ(ρ([1 2 3 4; ρ=(2, 2)])), ρ([1 2 3 4; ρ=(2, 2)]))
AsArrayWithShape(ρ(<2 2>), ρ([1 2 3 4; ρ=(2, 2)]))
AsArrayWithShape(<2>, ρ([1 2 3 4; ρ=(2, 2)]))
AsArrayWithValues(<2>, (<0> ψ ρ([1 2 3 4; ρ=(2, 2)])), (<1> ψ ρ([1 2 3 4; ρ=(2, 2)])))
AsArrayWithValues(<2>, (<0> ψ <2 2>), (<1> ψ ρ([1 2 3 4; ρ=(2, 2)])))
AsArrayWithValues(<2>, 2, (<1> ψ ρ([1 2 3 4; ρ=(2, 2)])))
AsArrayWithValues(<2>, 2, (<1> ψ <2 2>))
AsArrayWithValues(<2>, 2, 2)
[2 2; ρ=(2,)]
[2 2; ρ=(2,)]
[2 2; ρ=(2,)]


OK so far this has been pretty boring. Let's define a new array that is lazily computed and convert this to Python:

In [6]:
import matchpy

In [7]:
class SumIndex(matchpy.Symbol):
    def __init__(self, shape):
        self.shape = shape
        super().__init__(f"SumIndex({repr(shape)})", None)

In [8]:
sum_index = matchpy.Wildcard.symbol("sum_index", SumIndex)

In [9]:
uarray.register(uarray.Index(uarray.vector, sum_index), lambda vector, sum_index: uarray.Scalar(sum(vector.values)))
uarray.register(uarray.Shape(sum_index), lambda sum_index: uarray.Vector(*sum_index.shape))

Now let's see if can convert one of these to a concrete array:

In [10]:
expr = uarray.AsArray(SumIndex((2, 2, 2)))
print(expr)

AsArray(SumIndex((2, 2, 2)))


In [11]:
uarray.replace(expr)

Array('[0 1 1 2 1 2 2 3; ρ=(2, 2, 2)]')

Would be nice to print this better, but it seems right. let's take a look at the trace:

In [13]:
uarray.replace_debug(expr)

AsArray(SumIndex((2, 2, 2)))
AsArrayWithShape(ρ(SumIndex((2, 2, 2))), SumIndex((2, 2, 2)))
AsArrayWithShape(<2 2 2>, SumIndex((2, 2, 2)))
AsArrayWithValues(<2 2 2>, (<0 0 0> ψ SumIndex((2, 2, 2))), (<0 0 1> ψ SumIndex((2, 2, 2))), (<0 1 0> ψ SumIndex((2, 2, 2))), (<0 1 1> ψ SumIndex((2, 2, 2))), (<1 0 0> ψ SumIndex((2, 2, 2))), (<1 0 1> ψ SumIndex((2, 2, 2))), (<1 1 0> ψ SumIndex((2, 2, 2))), (<1 1 1> ψ SumIndex((2, 2, 2))))
AsArrayWithValues(<2 2 2>, 0, (<0 0 1> ψ SumIndex((2, 2, 2))), (<0 1 0> ψ SumIndex((2, 2, 2))), (<0 1 1> ψ SumIndex((2, 2, 2))), (<1 0 0> ψ SumIndex((2, 2, 2))), (<1 0 1> ψ SumIndex((2, 2, 2))), (<1 1 0> ψ SumIndex((2, 2, 2))), (<1 1 1> ψ SumIndex((2, 2, 2))))
AsArrayWithValues(<2 2 2>, 0, 1, (<0 1 0> ψ SumIndex((2, 2, 2))), (<0 1 1> ψ SumIndex((2, 2, 2))), (<1 0 0> ψ SumIndex((2, 2, 2))), (<1 0 1> ψ SumIndex((2, 2, 2))), (<1 1 0> ψ SumIndex((2, 2, 2))), (<1 1 1> ψ SumIndex((2, 2, 2))))
AsArrayWithValues(<2 2 2>, 0, 1, 1, (<0 1 1> ψ SumIndex((2, 2, 2))), (<1 0 0> ψ

You can see that it expands this out into indexing all values into the array, using it's shape to calculate what these should be.